In [12]:
os.chdir(r"C:\Users\Ahicks88\Documents\Analytics\GitHub\ContinuedLearning\DataCamp\Python\Python Programming\Analyzing US Census Data in Python\Datasets")

In [14]:
# Census Subject Tables:

states.head()

import seahborn as sns
sns.set()
sms.barplot(
    x = "total",
    y = states.index,
    data = states
)

NameError: name 'states' is not defined

In [ ]:
# Aggregate and Calculate Proportions:

# What percentage of Hispanics identify as White?
print(100 * states["hispanic_white"].sum() / states["hispanic"].sum())

# Set list of Hispanic race column names
hispanic_races = [
    "hispanic_white", 
    "hispanic_black", "hispanic_aian", 
    "hispanic_asian", "hispanic_pacific", 
    "hispanic_other", "hispanic_multiracial"
    ]

# What percentage of Hispanics identify as each race?
print(100 * states[hispanic_races].sum() / states["hispanic"].sum())